In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import seaborn as sns

In [74]:
df= pd.read_csv("https://raw.githubusercontent.com/Apress/data-analysis-and-visualization-using-python/master/Ch07/Salaries.csv")

In [85]:
df.to_csv('refined.csv', index=False)
df = pd.read_csv('refined.csv')

df.groupby(['rank'], sort=False, as_index=False)[['salary']].describe()


salary                                                                       \
   count           mean           std      min       25%       50%        75%   
0   46.0  123624.804348  24850.287853  57800.0  105112.5  123321.5  143512.25   
1   13.0   91786.230769  18571.183714  62884.0   74830.0  103613.0  104542.00   
2   19.0   81362.789474   9381.245301  63100.0   74096.0   78500.0   91150.00   

             
        max  
0  186960.0  
1  119800.0  
2   97032.0

In [171]:
#count and missing
count_missing = df.groupby(['rank'])[['salary']].apply(lambda x: x.isna().sum())

count_missing.columns=['Missing']

count_missing=count_missing.transpose()
count_missing

rank,AssocProf,AsstProf,Prof
Missing,0,0,0


In [172]:
#valid to std
majority = df.groupby(['rank'], sort=False)[['salary']].agg(['count','median','mean','sem','std',])
#gawa ako separate for valid and missing? then concat nalang? then pivot?

majority.columns=['Valid','Median','Mean','Std. error of Mean', 'Std. Deviation']

majority=majority.transpose()
majority

rank,Prof,AssocProf,AsstProf
Valid,46.000000,13.000000,19.000000
Median,123321.500000,103613.000000,78500.000000
Mean,123624.804348,91786.230769,81362.789474
Std. error of Mean,3663.975052,5150.719625,2152.205275
Std. Deviation,24850.287853,18571.183714,9381.245301


In [149]:
#coefficient of variation: std/mean

coef_Var = df.groupby(['rank'], sort=False)[['salary']].apply(lambda x: (x.std())/x.mean())
coef_Var.columns=['Coefficient of Variation']
coef_Var=coef_Var.transpose()
coef_Var

rank,Prof,AssocProf,AsstProf
Coefficient of Variation,0.201014,0.202331,0.115301


In [145]:
skewness = df.groupby(['rank'], sort=False)[['salary']].skew()

skewness.columns=['Skewness']

skewness=skewness.transpose()
skewness

rank,Prof,AssocProf,AsstProf
Skewness,0.070309,-0.1512,0.030504


In [185]:
n = df.groupby(['rank'], sort=False)[['salary']].count()
se_skewness = np.sqrt(6*n*(n-1) / ((n-2)*(n+1)*(n+3))) * (1 - ((1-3*n)**2 / ((n-2)*(n-3))))
se_skewness.columns=['Std. Error of Skewness']

se_skewness=se_skewness.transpose()

se_skewness

rank,Prof,AssocProf,AsstProf
Std. Error of Skewness,-3.122925,-7.474475,-5.514955


In [177]:
kurt = df.groupby(['rank'], sort=False)[['salary']].apply(lambda x : x.kurt())
kurt.columns=['Kurtosis']
kurt=kurt.transpose()

kurt

rank,Prof,AssocProf,AsstProf
Kurtosis,0.201339,-1.614149,-1.011154


In [212]:
se_kurtosis = ((24*n*(n-2)*(n-3)) / ((n+1)*(n+3)*(n+5)*(n+7)))**0.5

se_kurtosis.columns=['Std. Error of Kurtosis']
se_kurtosis=se_kurtosis.transpose()

se_kurtosis

rank,Prof,AssocProf,AsstProf
Std. Error of Kurtosis,0.579262,0.652377,0.672122


In [173]:
min_Max = df.groupby(['rank'], sort=False)[['salary']].agg(['min','max'])
min_Max.columns=['Minimum','Maximum']
min_Max = min_Max.transpose()

min_Max

rank,Prof,AssocProf,AsstProf
Minimum,57800,62884,63100
Maximum,186960,119800,97032


In [189]:
#quartiles == 
quarts = df.groupby(['rank'], sort=False)[['salary']].apply(lambda x:x.quantile([0.10,0.2,.25,.3,.4,.5,.6,.7,.75,.8,.9])).unstack()
quarts.columns=['10th Percentile','20th Percentile','25th Percentile','30th Percentile','40th Percentile','50th Percentile','60th Percentile','70th Percentile','75th Percentile','80th Percentile','90th Percentile']

quarts= quarts.transpose()
quarts

rank,Prof,AssocProf,AsstProf
10th Percentile,93675.00,71512.0,72500.0
20th Percentile,103275.00,73912.0,73300.0
25th Percentile,105112.50,74830.0,74096.0
30th Percentile,108473.00,76432.0,74832.8
40th Percentile,116450.00,80528.0,77100.0
50th Percentile,123321.50,103613.0,78500.0
60th Percentile,127512.00,103798.8,84045.0
70th Percentile,135889.00,104213.2,89800.0
75th Percentile,143512.25,104542.0,91150.0
80th Percentile,148750.00,106021.6,91580.0


In [213]:
order = [count_missing,majority,coef_Var, skewness, se_skewness, kurt, se_kurtosis, min_Max, quarts]
desc_Stats = pd.concat(order)
desc_Stats.columns= pd.MultiIndex.from_product([['Salary'],desc_Stats.columns])

desc_Stats

Salary                             
rank                          AssocProf      AsstProf           Prof
Missing                        0.000000      0.000000       0.000000
Valid                         13.000000     19.000000      46.000000
Median                    103613.000000  78500.000000  123321.500000
Mean                       91786.230769  81362.789474  123624.804348
Std. error of Mean          5150.719625   2152.205275    3663.975052
Std. Deviation             18571.183714   9381.245301   24850.287853
Coefficient of Variation       0.202331      0.115301       0.201014
Skewness                      -0.151200      0.030504       0.070309
Std. Error of Skewness        -7.474475     -5.514955      -3.122925
Kurtosis                      -1.614149     -1.011154       0.201339
Std. Error of Kurtosis         0.652377      0.672122       0.579262
Minimum                    62884.000000  63100.000000   57800.000000
Maximum                   119800.000000  97032.000000  186960.000000
10th Percentile            71512.000000  72500.000000   93675.000000
20th Percentile            73912.000000  73300.000000  103275.000000
25th Percentile            74830.000000  74096.000000  105112.500000
30th Percentile            76432.000000  74832.800000  108473.000000
40th Percentile            80528.000000  77100.000000  116450.000000
50th Percentile           103613.000000  78500.000000  123321.500000
60th Percentile           103798.800000  84045.000000  127512.000000
70th Percentile           104213.200000  89800.000000  135889.000000
75th Percentile           104542.000000  91150.000000  143512.250000
80th Percentile           106021.600000  91580.000000  148750.000000
90th Percentile           109121.600000  92000.000000  154750.000000